In [21]:
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

from gensim.utils import simple_preprocess
import pyLDAvis
import pyLDAvis.gensim_models
# Visualize the topics
pyLDAvis.enable_notebook()
import pickle 
from pprint import pprint
import re
import os
import numpy as np
import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
import spacy

In [3]:
## In resource or connectivity-constrained environments, consider saving these off as a text file or whatever

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Data
- https://www.kaggle.com/datasets/fabiochiusano/medium-articles?resource=download
### LDA
- https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0  
### Topic Coherence
- https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0  
### Tokenization help
- https://towardsdatascience.com/5-simple-ways-to-tokenize-text-in-python-92c6804edfc4  

In [7]:
df = pd.read_csv(r"D:\Work\Data\medium_articles.csv", nrows=5000)

In [8]:
df

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."
...,...,...,...,...,...,...
4995,"NLP visualizations for clear, immediate insigh...","NLP visualizations for clear, immediate insigh...",https://medium.com/plotly/nlp-visualisations-f...,['Jp Hwang'],2020-03-30 16:37:04.161000+00:00,"['Data Science', 'Programming', 'Python', 'Mac..."
4996,Swift — Creating a Custom View From a XIB (Upd...,9. Use Your View\n\nWe’re done! Now we’ve got ...,https://medium.com/better-programming/swift-3-...,['Brian Clouser'],2019-07-02 23:03:29.913000+00:00,"['iOS', 'Swift', 'Mobile App Development', 'Sw..."
4997,Can You Solve The Mystery Of Edgar Allan Poe’s...,Can You Solve The Mystery Of Edgar Allan Poe’s...,https://medium.com/omgfacts/can-you-solve-the-...,['New Visions'],2016-09-23 22:05:12.751000+00:00,"['Edgar Allan Poe', 'Mind', 'Poetry', 'Crime',..."
4998,Your Life Is Full of Porn. Stop Getting Yourse...,This Is Your Life On Porn\n\nYou wake up. You ...,https://medium.com/the-ascent/your-life-is-ful...,['Tim Denning'],2020-07-31 17:01:01.525000+00:00,"['Addiction', 'Life', 'Money', 'Social Media',..."


In [9]:
# Remove punctuation
df['text_processed'] = df['text'].map(lambda x: re.sub('[,\.!?]-_(){}', '', x))
# Convert the titles to lowercase
df['text_processed'] = df['text_processed'].map(lambda x: x.lower())

df['text_processed'] = df['text_processed'].str.replace("\n", " ")
df['text_processed'] = df['text_processed'].str.replace("\t", " ")
df['text_processed'] = df['text_processed'].str.replace(pat=r"\s{2,}", repl=" ", regex=True)

df['text_processed'] = df['text_processed'].map(lambda x: " ".join([word for word in x.split(' ') if word.isalnum()]))
df['text_processed'] = df['text_processed'].map(lambda x: " ".join([word for word in x.split(' ') if len(word)>2]))

In [10]:
df['text_processed']

0       photo josh riemer unsplash merry christmas and...
1       your brain coronavirus guide the curious and t...
2       mind your nose how smell training can change y...
3       passionate about the synergy between science a...
4       heard phineas the railroad worker who survived...
                              ...                        
4995    nlp visualizations for immediate insights into...
4996    use your view now got custom made from xib tha...
4997    can you solve the mystery edgar allan the evid...
4998    this your life porn you wake you the toilet an...
4999    with the growth the amount data available pres...
Name: text_processed, Length: 5000, dtype: object

In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

## Added this from the topic coherence article

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
data = df.text_processed.values.tolist()
data_words = list(sent_to_words(data))
# Remove Stop Words
# data_words = remove_stopwords(data_words)
data_words_nostops = remove_stopwords(data_words)

print(data_words[:1][0][:30])

['photo', 'josh', 'riemer', 'unsplash', 'merry', 'christmas', 'happy', 'wanted', 'everyone', 'know', 'much', 'appreciate', 'everyone', 'thankful', 'readers', 'writers', 'anywhere', 'without', 'thank', 'bringing', 'important', 'pieces', 'destigmatize', 'mental', 'illness', 'mental', 'without', 'ten', 'top', 'stories']


## Optional: Make Bigrams and Trigrams

In [13]:
# Build the bigram model
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
# Faster way to get a sentence clubbed as a bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [ ]:
# Build the trigram model
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [15]:


# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)


[['happy', 'want', 'know', 'much', 'appreciate', 'thankful', 'reader', 'writer', 'anywhere', 'thank', 'bring', 'important', 'piece', 'destigmatize', 'mental_illness', 'mental', 'top', 'story', 'last', 'capacity', 'love', 'inspire', 'universal', 'capacity', 'hate', 'irrespective', 'age', 'religion', 'none', 'exempt', 'aggressive', 'accordingly', 'repress', 'deep', 'seat', 'feeling', 'inferiority', 'inflate', 'delusion', 'grandeur', 'prone', 'aggression', 'infiltrate', 'interaction', 'myriad', 'environment', 'school', 'cyber', 'bullying', 'happen', 'ringleader', 'look', 'minion', 'sanction', 'cruelty', 'look', 'circumstance', 'bring', 'sad', 'grateful', 'program', 'change', 'life', 'help', 'imagine', 'life', 'like', 'learn', 'accept', 'powerlessness', 'prioritize', 'take', 'life', 'step', 'never', 'bet', 'world', 'much', 'prospect', 'spending', 'horrible', 'locked', 'psychiatric', 'low', 'point', 'day', 'room', 'festoon', 'cheesy', 'decoration', 'sorry', 'pink', 'aluminum', 'therapy', 'r

In [ ]:
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

In [17]:

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1)]


In [18]:

# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.012*"business" + 0.010*"company" + 0.010*"get" + 0.009*"make" + '
  '0.008*"marketing" + 0.007*"people" + 0.007*"content" + 0.006*"new" + '
  '0.006*"email" + 0.006*"create"'),
 (1,
  '0.011*"design" + 0.009*"make" + 0.008*"user" + 0.008*"product" + '
  '0.008*"work" + 0.007*"team" + 0.007*"new" + 0.007*"app" + 0.006*"need" + '
  '0.006*"use"'),
 (2,
  '0.015*"get" + 0.011*"make" + 0.011*"time" + 0.010*"work" + 0.009*"take" + '
  '0.009*"know" + 0.008*"people" + 0.008*"thing" + 0.007*"go" + 0.007*"feel"'),
 (3,
  '0.018*"use" + 0.010*"create" + 0.009*"code" + 0.009*"image" + 0.009*"model" '
  '+ 0.008*"function" + 0.008*"datum" + 0.006*"value" + 0.006*"make" + '
  '0.006*"add"'),
 (4,
  '0.009*"people" + 0.007*"study" + 0.007*"also" + 0.006*"health" + '
  '0.005*"case" + 0.005*"get" + 0.005*"virus" + 0.004*"say" + 0.004*"many" + '
  '0.004*"make"'),
 (5,
  '0.010*"human" + 0.008*"datum" + 0.006*"make" + 0.006*"use" + 0.005*"system" '
  '+ 0.005*"also" + 0.004*"brain" + 0.004*


Coherence Score:  0.39647229659125294


In [ ]:

for i in (10,20,30,50):
    # number of topics
    num_topics = i
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True)
    # Print the Keyword in the 10 topics
    pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)
    print("\n\n ------------ \n\n")

[(0,
  '0.011*"business" + 0.009*"get" + 0.009*"make" + 0.009*"company" + '
  '0.008*"marketing" + 0.007*"people" + 0.007*"content" + 0.006*"new" + '
  '0.006*"email" + 0.006*"create"'),
 (1,
  '0.010*"design" + 0.009*"make" + 0.009*"user" + 0.008*"product" + '
  '0.008*"work" + 0.008*"team" + 0.007*"new" + 0.007*"need" + 0.007*"app" + '
  '0.006*"help"'),
 (2,
  '0.015*"get" + 0.011*"make" + 0.011*"time" + 0.010*"work" + 0.009*"take" + '
  '0.009*"know" + 0.008*"people" + 0.008*"thing" + 0.007*"go" + 0.007*"feel"'),
 (3,
  '0.018*"use" + 0.011*"create" + 0.009*"datum" + 0.009*"model" + 0.008*"code" '
  '+ 0.007*"image" + 0.007*"function" + 0.006*"make" + 0.006*"need" + '
  '0.006*"set"'),
 (4,
  '0.009*"people" + 0.007*"study" + 0.006*"also" + 0.006*"health" + '
  '0.005*"get" + 0.005*"virus" + 0.004*"case" + 0.004*"many" + 0.004*"make" + '
  '0.004*"say"'),
 (5,
  '0.010*"human" + 0.009*"idea" + 0.008*"datum" + 0.006*"make" + 0.005*"use" + '
  '0.005*"brain" + 0.005*"also" + 0.005*"p


Coherence Score:  0.3954825088041496


 ------------ 


[(7,
  '0.038*"datum" + 0.016*"value" + 0.014*"use" + 0.012*"plot" + '
  '0.010*"dataset" + 0.010*"number" + 0.009*"feature" + 0.009*"point" + '
  '0.009*"column" + 0.008*"see"'),
 (26,
  '0.024*"service" + 0.013*"use" + 0.011*"application" + 0.011*"datum" + '
  '0.010*"run" + 0.008*"kubernete" + 0.008*"create" + 0.008*"log" + '
  '0.007*"need" + 0.007*"provide"'),
 (32,
  '0.009*"make" + 0.008*"people" + 0.008*"get" + 0.007*"know" + 0.007*"feel" + '
  '0.007*"think" + 0.007*"see" + 0.006*"take" + 0.006*"life" + 0.006*"even"'),
 (40,
  '0.016*"feel" + 0.016*"get" + 0.012*"work" + 0.012*"day" + 0.010*"make" + '
  '0.010*"time" + 0.010*"people" + 0.008*"know" + 0.007*"thing" + '
  '0.007*"sleep"'),
 (15,
  '0.017*"dog" + 0.016*"get" + 0.009*"make" + 0.009*"people" + 0.008*"start" + '
  '0.008*"path" + 0.007*"video" + 0.007*"play" + 0.006*"conda" + 0.006*"see"'),
 (24,
  '0.020*"dough" + 0.016*"end" + 0.015*"pizza" + 0.009*"chef" + 

In [25]:
len(corpus)

5000

In [20]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [26]:

grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = int(len(corpus))
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [27]:
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()


  0%|                                                                                          | 0/540 [05:40<?, ?it/s]


ValueError: Stop argument for islice() must be None or an integer: 0 <= x <= sys.maxsize.

In [ ]:
# LDAvis_data_filepath = os.getcwd()+

LDAvis_data_filepath = os.path.join(f'{os.getcwd()}/results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

In [ ]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, f'{os.getcwd()}/results/ldavis_prepared_'+ str(num_topics) +'.html')

In [ ]:
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

In [ ]:
pyLDAvis.save_html(LDAvis_prepared, f'ldavis_prepared_{str(num_topics)}.html')

In [ ]:
LDAvis_prepared